In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')

In [ ]:
def weird_division(n, d):
    return n / d if d else 0
df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
df.head()

In [ ]:
df.drop(['total_revenue', 'revenue_share_percent', 'integration_type_id'], axis = 1, inplace=True)
df = df[df['CPM'] >= 0]
df_train = df[df['date'] < '2019-06-22']
df_test = df[df['date'] >= '2019-06-22']

In [ ]:
df_train.describe()

In [ ]:
corr = df_train.corr()
plt.figure(figsize=(12,8))
sns.heatmap(data=corr,vmin=0, vmax=1, cmap='coolwarm',  square=True)
plt.show()

In [ ]:
df_train = df_train[df_train['CPM'] < df_train['CPM'].quantile(0.95)]
df_test = df_test[df_test['CPM'] < df_test['CPM'].quantile(0.95)]
X_train, y_train = df_train.drop(columns = ['CPM', 'date']), df_train['CPM']
X_test, y_test = df_test.drop(columns = ['CPM', 'date']), df_test['CPM']

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

In [ ]:
predicted = model.predict(X_test)
print("MSE: ", round(mean_squared_error(predicted, y_test), 4))